In [1]:
# 將所有 csv 讀入同一個 df 入面
import pandas as pd
import glob

all_files = glob.glob("./input/*.csv")
file_list = []
for file in all_files:
    df = pd.read_csv(file, header=None, names=["label", "word", "jyutping"], index_col=False)
    file_list.append(df)

data = pd.concat(file_list, axis=0, ignore_index=True).drop_duplicates()
data.drop_duplicates(inplace=True)
del df, file, all_files, file_list

### 第二次標註執漏

讀晒所有數據之後，按照下面嘅規則提取有用嘅字詞出嚟

1. 所有標咗 1* 嘅詞條單獨列出嚟保留成`error_words`
1. 剩低嘅詞條入面，按照有冇粵拼同有冇標 1 分成四個文件

對於呢四個文件，**`jyutping_useful`係肯定保留嘅**，`no_jyutping_junk` 係肯定要鏟晒嘅。而 `jyutping_junk` 同埋 `no_jyutping_useful`就有疑問，需要攞出嚟畀標註人再標多次。

In [2]:
# 需要人手屋修改嘅詞彙
error_words = data[data["label"] == "1*"]

remain = data[data["label"] != "1*"]

# 有粵拼，全部都保留嘅詞彙
has_jyutping = remain[remain["jyutping"].notna()]
no_jyutping = remain[remain["jyutping"].isna()]


# 冇粵拼，但係人工揀出嚟有用嘅詞彙
jyutping_useful = has_jyutping[has_jyutping["label"].notna()]
jyutping_junk = has_jyutping[has_jyutping["label"].isna()]
no_jyutping_useful = no_jyutping[no_jyutping["label"].notna()]
no_jyutping_junk = no_jyutping[no_jyutping["label"].isna()]

del remain, has_jyutping, no_jyutping

error_words.sort_values(by=["jyutping"], inplace=True)
error_words.to_csv("output/error_words.csv", index=False, columns=["word", "jyutping"])

jyutping_useful.sort_values(by=["jyutping"], inplace=True)
jyutping_useful.to_csv("output/jyutping_useful.csv", index=False, columns=["word", "jyutping"])

jyutping_junk.sort_values(by=["jyutping"], inplace=True)
jyutping_junk.to_csv("output/jyutping_junk.csv", index=False, columns=["word", "jyutping"])

no_jyutping_useful.sort_values(by=["jyutping"], inplace=True)
no_jyutping_useful.to_csv("output/no_jyutping_useful.csv", index=False, columns=["word", "jyutping"])

no_jyutping_junk.sort_values(by=["jyutping"], inplace=True) 
no_jyutping_junk.to_csv("output/no_jyutping_junk.csv", index=False, columns=["word", "jyutping"])

C:\Users\liuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)



分成每個文件 1000 個詞畀標註人員標



In [5]:
# 然後要將 jyutping_junk 同埋 no_jyutping_useful 分出嚟
reannotation = jyutping_junk.append(no_jyutping_useful)
shuffled = reannotation.sample(frac=1)

num_slices = shuffled.shape[0] // 1000 + 1

for i in range(num_slices):
    start = i * 1000
    end = (i + 1) * 1000
    sliced = shuffled.iloc[start:end]
    sliced.to_csv(f'./reannotate/r1r_{i+1}.csv', index=False, columns=["word", "jyutping"])

因為喺今次標註入面，有粵拼嗰啲之前都係被當成垃圾嘅，冇粵拼嗰啲之前都係畀人當成有用嘅，所以今次標註入面 `jyutping_junk` 肯定可以剷走，因為兩次都畀人當成垃圾。而 `no_jyutping_useful` 肯定可以保留，因為兩次都畀人當有用。所以今次嘅`no_jyutping_useful`同埋上次嘅`jyutping_useful`兩個文件就變成咗最尾嘅輸出底。

而 `jyutping_useful` 同埋 `no_jyutping_junk`就需要畀人人工檢查嘞



In [1]:
# 將所有 csv 讀入同一個 df 入面
import pandas as pd
import glob

all_files = glob.glob("./re_input/*.csv")
file_list = []
for file in all_files:
    df = pd.read_csv(file, header=None, names=["label", "word", "jyutping"], index_col=False)
    file_list.append(df)

data = pd.concat(file_list, axis=0, ignore_index=True).drop_duplicates()
data.drop_duplicates(inplace=True)
del df, file, all_files, file_list

# 需要人手屋修改嘅詞彙
error_words = data[data["label"] == "1*"]

remain = data[data["label"] != "1*"]

# 有粵拼，全部都保留嘅詞彙
has_jyutping = remain[remain["jyutping"].notna()]
no_jyutping = remain[remain["jyutping"].isna()]


# 冇粵拼，但係人工揀出嚟有用嘅詞彙
jyutping_useful = has_jyutping[has_jyutping["label"].notna()]
jyutping_junk = has_jyutping[has_jyutping["label"].isna()]
no_jyutping_useful = no_jyutping[no_jyutping["label"].notna()]
no_jyutping_junk = no_jyutping[no_jyutping["label"].isna()]

del remain, has_jyutping, no_jyutping

error_words.sort_values(by=["jyutping"], inplace=True)
error_words.to_csv("re_output/error_words.csv", index=False, columns=["word", "jyutping"])

jyutping_useful.sort_values(by=["jyutping"], inplace=True)
jyutping_useful.to_csv("re_output/jyutping_useful.csv", index=False, columns=["word", "jyutping"])

jyutping_junk.sort_values(by=["jyutping"], inplace=True)
jyutping_junk.to_csv("re_output/jyutping_junk.csv", index=False, columns=["word", "jyutping"])

no_jyutping_useful.sort_values(by=["jyutping"], inplace=True)
no_jyutping_useful.to_csv("re_output/no_jyutping_useful.csv", index=False, columns=["word", "jyutping"])

no_jyutping_junk.sort_values(by=["jyutping"], inplace=True) 
no_jyutping_junk.to_csv("re_output/no_jyutping_junk.csv", index=False, columns=["word", "jyutping"])

C:\Users\liuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## 下一步操作

而家總共有呢啲數據係確定可用嘅：

1. `output/jyutping_useful.csv`
1. `re_output/no_jyutping_useful.csv`
1. `re_output/jyutping_useful.csv`

而下面文件係需要加粵拼然後人手檢查嘅

1. `re_output/no_jyutping_useful.csv`

跟住下面文件係需要人手修正嘅：
1. `output/error_words.csv`
1. `re_output/error_words.csv`


In [14]:
from ToJyutping import get_jyutping_text

no_jyutping_useful = pd.read_csv("re_output/no_jyutping_useful.csv")
no_jyutping_useful["jyutping"] = no_jyutping_useful["word"].apply(get_jyutping_text)
no_jyutping_useful.to_csv("to_check_jyutping.csv", index=False, columns=["word", "jyutping"])